Load the modules

In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from src import datareader, enums

Load in the data and split between training and testing

In [2]:
filename = '/Users/mhogan/Documents/heartfailurepred/heart.csv'
keys, features, labels = datareader.get_csv_features_labels(filename)
tt_splits = train_test_split(features, labels, test_size=0.33)
features_train, features_test, labels_train, labels_test = tt_splits

Normalize the data

In [3]:
normalized_features_train = preprocessing.normalize(features_train.swapaxes(0, 1)).swapaxes(0, 1)
normalized_features_test = preprocessing.normalize(features_test.swapaxes(0, 1)).swapaxes(0, 1)
normalized_features_test = normalized_features_test.reshape((len(normalized_features_test), len(keys)-1))

Define the NN model

In [4]:
n_epochs = 1000

# Single hidden layer NN
heart_disease_model = tf.keras.Sequential([
    layers.Dense(len(keys) * 2),
    layers.Dense(1)
])

# Compile the model and fit
heart_disease_model.compile(loss=tf.losses.BinaryCrossentropy(),
                            optimizer=tf.optimizers.Adam())
heart_disease_model.fit(normalized_features_train, labels_train, epochs=n_epochs)

2021-10-10 13:48:43.665218: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-10 13:48:43.787584: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/1000
20/20 [==============================] - 1s 2ms/step - loss: 8.4580
Epoch 2/1000
20/20 [==============================] - 0s 2ms/step - loss: 8.3938
Epoch 3/1000
20/20 [==============================] - 0s 2ms/step - loss: 8.0914
Epoch 4/1000
20/20 [==============================] - 0s 2ms/step - loss: 8.5871
Epoch 5/1000
20/20 [==============================] - 0s 2ms/step - loss: 8.2620
Epoch 6/1000
20/20 [==============================] - 0s 2ms/step - loss: 8.2833
Epoch 7/1000
20/20 [==============================] - 0s 2ms/step - loss: 8.2364
Epoch 8/1000
20/20

Predict the results

In [5]:
predictions_class = heart_disease_model.predict_classes(normalized_features_test).swapaxes(0, 1).flatten()

/Users/mhogan/.conda/envs/heartfailurepred/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Calculate intermediate results

In [7]:
true_classes = enums.HeartDiseaseClassification()
true_positive = np.count_nonzero(np.logical_and(predictions_class == true_classes.HeartDisease,
                                                labels_test == true_classes.HeartDisease).astype(int))
false_positive = np.count_nonzero(np.logical_and(predictions_class == true_classes.HeartDisease,
                                                 labels_test == true_classes.Normal).astype(int))
false_negative = np.count_nonzero(np.logical_and(predictions_class == true_classes.Normal,
                                                 labels_test == true_classes.HeartDisease).astype(int))
recall = true_positive / (false_negative + true_positive)
precision = true_positive / (true_positive + false_positive)
true_negative = np.count_nonzero(np.logical_and(predictions_class == true_classes.Normal,
                                                labels_test == true_classes.Normal).astype(int))
positives = np.count_nonzero((labels_test == true_classes.HeartDisease).astype(int))
negatives = len(labels_test) - positives

ZeroDivisionError: division by zero

Calculate metrics

In [ ]:
sensitivity = float(true_positive) / positives
specificity = float(true_negative) / negatives
accuracy = (true_positive + true_negative) / (positives + negatives)
f1_score = 2. / (1./recall + 1./precision)
mcc = (true_positive * true_negative) - (false_positive * false_negative)
mcc /= np.sqrt((true_positive + false_positive) * (true_positive + false_negative) *
               (true_negative + false_positive) * (true_negative + false_negative))
print('accuracy = ', accuracy)
print('sensitivity = ', sensitivity)
print('specificity = ', specificity)
print('f1_score = ', f1_score)
print('Matthew\'s Corrleation Coefficient = ', mcc)
